# Prerequisites

In [ ]:
import numpy as np
import pandas as pd
from tm import StockDataProvider
from tm.trading_rules import SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC
from tm.optimizers import GeneticOptimizer, StrategyPerformanceEvaluator, map_chromosome_to_trading_rule_parameters, filter_for_active_rules, calculate_absolute_buy_and_hold_returns
from tm.backtesting import GeometricBrownianMotion, MonteCarloCrossValidation
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

Define the trading rule basis for the strategy.

In [ ]:
trading_rules = [SimpleMovingAverage, ExponentialMovingAverage, STO, MACD, RSI, ROC]

Define a function which finds the best strategy by running the genetic algorithm and applying backtesting.

In [ ]:
# Returns best strategy as bitvector
def find_best_strategy(ticker_name, start, end, ngen=5):
    data = StockDataProvider(ticker_name, start=start, end=end)
    optimizer = GeneticOptimizer(data, trading_rules, random_seed=42)
    print('Running genetic algorithm ...')
    hof = optimizer.run(pop_size=1000, ngen=ngen, hof_size=100)
    print('Net profit of buy and hold:', calculate_absolute_buy_and_hold_returns(data))
    gbm = GeometricBrownianMotion(data)
    crossvalidator = MonteCarloCrossValidation(hof, gbm, trading_rules)
    print('Running backtesting ...')
    best_individual = crossvalidator.run(num_iterations=100, time_steps=85)
    del optimizer
    return best_individual

Define a function which prints the results for the testing period.

In [ ]:
def print_test_results(individual, ticker_name, start, end):
    real_data = StockDataProvider(ticker_name, start=start, end=end)
    rule_instances = list(map(lambda Rule, params: Rule(real_data, *params), trading_rules, map_chromosome_to_trading_rule_parameters(individual, trading_rules)))
    active_rule_instances = filter_for_active_rules(individual, rule_instances)
    evaluator = StrategyPerformanceEvaluator(active_rule_instances)
    net_profit = evaluator.calculate_net_profit()
    net_profit_bh = calculate_absolute_buy_and_hold_returns(real_data)
    percentage_profit = net_profit / real_data.history['Close'].iloc[0] * 100
    percentage_profit_bh = net_profit_bh / real_data.history['Close'].iloc[0] * 100
    print('Active rules:', list(map(lambda rule: rule.__class__.__name__, active_rule_instances)))
    print('Active rules\' parameters:', list(map(lambda param: param[1], filter(lambda param: trading_rules[param[0]] in map(lambda rule: rule.__class__, active_rule_instances), enumerate(map_chromosome_to_trading_rule_parameters(individual, trading_rules))))))
    print('Net profit of strategy: {:f}'.format(net_profit))
    print('Net profit of buy and hold: {:f}'.format(net_profit_bh))
    print('Percentage profit of strategy: {:f}%'.format(percentage_profit))
    print('Percentage profit of buy and hold: {:f}%'.format(percentage_profit_bh))
    print('Number of buy signals:', len(evaluator.buy_signals[evaluator.buy_signals == True]))
    print('Number of sell signals:', len(evaluator.sell_signals[evaluator.sell_signals == True]))
    return evaluator, percentage_profit, percentage_profit_bh

Define a function which can plot a strategy.

In [ ]:
def plot_strategy(evaluator, ticker_name, start, end):
    data = StockDataProvider(ticker_name, start=start, end=end)
    
    fig, ax = plt.subplots(figsize=(16, 8))
    
    ax.plot(data.history['Close'], label='Stock prices')

    if len(evaluator.buy_signals[evaluator.buy_signals == True]) > 0:
        buy_points = pd.Series(data=map(lambda index, price: price if evaluator.buy_signals.loc[index] == True else np.nan, data.history['Close'].index, data.history['Close']),
                               index=data.history['Close'].index)
        ax.scatter(x=buy_points.index, y=buy_points, color='green', marker='^', label='Buy signal')
    
    if len(evaluator.sell_signals[evaluator.sell_signals == True]) > 0:
        sell_points = pd.Series(data=map(lambda index, price: price if evaluator.sell_signals.loc[index] == True else np.nan, data.history['Close'].index, data.history['Close']),
                               index=data.history['Close'].index)
        ax.scatter(x=sell_points.index, y=sell_points, color='red', marker='v', label='Sell signal')

    ax.set_xlabel('Date')
    ax.set_ylabel('Stock price')
    ax.legend()
    fig.tight_layout()

# Experiments

In [ ]:
results = {'EXS1.DE': [],
           'ADS.DE': [],
           'ALV.DE': [],
           'BAS.DE': [],
           'DBK.DE': [],
           'DTE.DE': [],
           'SAP.DE': [],
           'DJI': [],
           'CSCO': [],
           'DD': [],
           'JPM': [],
           'MSFT': [],
           'NKE': [],
           'UNH': []}

## DAX

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2015-04-01', end='2015-07-30')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2015-09-28', end='2016-01-26')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2016-03-26', end='2016-07-24')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2016-09-22', end='2017-01-20')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='EXS1.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='EXS1.DE', start='2017-03-21', end='2017-07-19')
results['EXS1.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='EXS1.DE', start='2017-03-21', end='2017-07-19')

## Adidas

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2015-04-01', end='2015-07-30')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2015-09-28', end='2016-01-26')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2016-03-26', end='2016-07-24')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2016-09-22', end='2017-01-20')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ADS.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ADS.DE', start='2017-03-21', end='2017-07-19')
results['ADS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ADS.DE', start='2017-03-21', end='2017-07-19')

## Allianz

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2015-04-01', end='2015-07-30')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2015-09-28', end='2016-01-26')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2016-03-26', end='2016-07-24')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2016-09-22', end='2017-01-20')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='ALV.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='ALV.DE', start='2017-03-21', end='2017-07-19')
results['ALV.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='ALV.DE', start='2017-03-21', end='2017-07-19')

## BASF

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2015-04-01', end='2015-07-30')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2015-09-28', end='2016-01-26')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2016-03-26', end='2016-07-24')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2016-09-22', end='2017-01-20')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='BAS.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='BAS.DE', start='2017-03-21', end='2017-07-19')
results['BAS.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='BAS.DE', start='2017-03-21', end='2017-07-19')

## Deutsche Bank

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2015-04-01', end='2015-07-30')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2015-09-28', end='2016-01-26')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2016-03-26', end='2016-07-24')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2016-09-22', end='2017-01-20')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DBK.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DBK.DE', start='2017-03-21', end='2017-07-19')
results['DBK.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DBK.DE', start='2017-03-21', end='2017-07-19')

## Deutsche Telekom

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2015-04-01', end='2015-07-30')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2015-09-28', end='2016-01-26')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2016-03-26', end='2016-07-24')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2016-09-22', end='2017-01-20')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DTE.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DTE.DE', start='2017-03-21', end='2017-07-19')
results['DTE.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DTE.DE', start='2017-03-21', end='2017-07-19')

## SAP

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2015-04-01', end='2015-07-30')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2015-09-28', end='2016-01-26')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2016-03-26', end='2016-07-24')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2016-09-22', end='2017-01-20')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='SAP.DE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='SAP.DE', start='2017-03-21', end='2017-07-19')
results['SAP.DE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='SAP.DE', start='2017-03-21', end='2017-07-19')

## Dow Jones Index

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2015-04-01', end='2015-07-30')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2015-09-28', end='2016-01-26')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2016-03-26', end='2016-07-24')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2016-09-22', end='2017-01-20')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DJI', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DJI', start='2017-03-21', end='2017-07-19')
results['DJI'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DJI', start='2017-03-21', end='2017-07-19')

## Cisco

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2015-04-01', end='2015-07-30')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2015-09-28', end='2016-01-26')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2016-03-26', end='2016-07-24')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2016-09-22', end='2017-01-20')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='CSCO', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='CSCO', start='2017-03-21', end='2017-07-19')
results['CSCO'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='CSCO', start='2017-03-21', end='2017-07-19')

## DowDuPont

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2015-04-01', end='2015-07-30')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2015-09-28', end='2016-01-26')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2016-03-26', end='2016-07-24')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2016-09-22', end='2017-01-20')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='DD', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='DD', start='2017-03-21', end='2017-07-19')
results['DD'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='DD', start='2017-03-21', end='2017-07-19')

## JPMorgan Chase

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2015-04-01', end='2015-07-30')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2015-09-28', end='2016-01-26')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2016-03-26', end='2016-07-24')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2016-09-22', end='2017-01-20')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='JPM', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='JPM', start='2017-03-21', end='2017-07-19')
results['JPM'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='JPM', start='2017-03-21', end='2017-07-19')

## Microsoft

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2015-04-01', end='2015-07-30')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2015-09-28', end='2016-01-26')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2016-03-26', end='2016-07-24')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2016-09-22', end='2017-01-20')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='MSFT', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='MSFT', start='2017-03-21', end='2017-07-19')
results['MSFT'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='MSFT', start='2017-03-21', end='2017-07-19')

## Nike

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2015-04-01', end='2015-07-30')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2015-09-28', end='2016-01-26')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2016-03-26', end='2016-07-24')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2016-09-22', end='2017-01-20')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='NKE', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='NKE', start='2017-03-21', end='2017-07-19')
results['NKE'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='NKE', start='2017-03-21', end='2017-07-19')

## UnitedHealth

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2014-11-05', end='2015-03-31', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2015-04-01', end='2015-07-30')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2015-04-01', end='2015-07-30')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2015-05-07', end='2015-09-27', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2015-09-28', end='2016-01-26')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2015-09-28', end='2016-01-26')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2015-10-31', end='2016-03-25', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2016-03-26', end='2016-07-24')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2016-03-26', end='2016-07-24')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2016-04-30', end='2016-09-21', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2016-09-22', end='2017-01-20')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2016-09-22', end='2017-01-20')

In [ ]:
%%time
best_individual = find_best_strategy(ticker_name='UNH', start='2016-10-25', end='2017-03-20', ngen=5)

In [ ]:
evaluator, percentage_profit, percentage_profit_bh = print_test_results(best_individual, ticker_name='UNH', start='2017-03-21', end='2017-07-19')
results['UNH'].append([percentage_profit, percentage_profit_bh])

In [ ]:
plot_strategy(evaluator, ticker_name='UNH', start='2017-03-21', end='2017-07-19')